In [1]:
import pandas as pd
import numpy as np
import unicodedata

# Data

In [2]:
data = pd.read_csv('data/anzahl-sbb-bahnhofbenutzer.csv') #contains origin 
data_pop = pd.read_excel('data/data_pop.xlsx') #population for each city
data_red = pd.read_excel('data/data_pop_modif.xlsx') # population and origin. it does not contain zurich and geneve

In [3]:
data_split = data['Bahnhof_Gare_Stazione;Unité;Jahr;Anzahl Bahnhofbenutzer'].str.split(';', expand=True)
data_split.columns = ['city', 'Unité', 'Jahr', 'origin']

## Data Manipulation

In [4]:
data_2024 = data_split[(data_split['Unité']== 'DP/jour ouvré') & (data_split['Jahr']=='2024')]
data_2024 = data_2024.drop(['Unité', 'Jahr'], axis=1)

In [5]:
data_2024.loc[:, 'origin']  = pd.to_numeric(data_2024['origin'], errors='coerce')

In [6]:
to_remove = ['Uster', 'Zürich Stadelhofen']
data_2024 = data_2024[~data_2024['city'].isin(to_remove)]
data_2024 = data_2024.reset_index(drop=True)

We have to manage the different station in the city of Zürich and Geneve. To do that, first we aggregate the origin for this station in only one. 

In [7]:
df = data_2024

cities_to_merge_zh = ["Zürich HB",'Zürich Enge','Zürich Hardbrücke', 'Zürich Altstetten' ]
cities_to_merge_ge = ['Genève', 'Genève-Eaux-Vives']

mask_zh = df["city"].isin(cities_to_merge_zh)
mask_ge = df["city"].isin(cities_to_merge_ge)

merged_origin_zh = df.loc[mask_zh, "origin"].sum()
merged_origin_ge = df.loc[mask_ge, "origin"].sum()


new_row_zh = pd.DataFrame({"city": ["Zürich HB"], "origin": [merged_origin_zh]})
new_row_ge = pd.DataFrame({"city": ["Genève"], "origin": [merged_origin_ge]})


df_filtered = df[~mask_zh]
df_filtered = df_filtered[~mask_ge]
df_filtered_zh = pd.concat([df_filtered, new_row_zh], ignore_index=True)
df_final = pd.concat([df_filtered_zh, new_row_ge], ignore_index=True)
df_final

C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_25872\2249174985.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered = df_filtered[~mask_ge]


,city,origin
0,Neuchâtel,28900
1,Thun,40400
2,Bellinzona,16100
3,Lugano,34100
4,St. Gallen,77400
5,Winterthur,134700
6,Zug,58900
7,Bern,298900
8,Luzern,145400
9,Baden,55400


# Estimate the destination

As we have only the origin we try to estiamte the destinatio in order to utilise a gravity model and define a origin-destiantion matrix. 

To estimate the destination we consider the population and the origin of each cities and we compute a weight. The weight reppresent the power of the city with respect to the others, the weight are between 0 and 1. If the weigth w --> 1 means that the cities attract more people.

In [8]:
total_origin = df_final['origin'].sum()
population_total = data_pop['habitant'].sum()


menage the station in geneve

In [9]:
geneve_stations = ['Genève', 'Genève-Aéroport']  
df_ge = df_final.loc[df_final['city'].isin(geneve_stations)].copy() 

pop_ge = data_pop.loc[data_pop['City'] == 'Genève'].copy()
total_origin_ge = df_ge['origin'].sum()

df_ge['weight'] = df_ge['origin'] / total_origin_ge
population_weight_ge = (pop_ge['habitant'] + total_origin_ge) / (total_origin + population_total)
df_ge

,city,origin,weight
15,Genève-Aéroport,43000,0.18647
20,Genève,187600,0.81353


In [10]:
df_ge['weight_population'] = df_ge['weight']*population_weight_ge[1]
df_ge['destination'] = df_ge['weight_population']*total_origin
df_ge


,city,origin,weight,weight_population,destination
15,Genève-Aéroport,43000,0.18647,0.019556,45734.763171
20,Genève,187600,0.81353,0.085321,199531.199324


menage the stations of zurich

In [11]:
zurich_station = ['Zürich Oerlikon', 'Zürich HB']
df_zh = df_final[df_final['city'].isin(zurich_station)].copy()

pop_zh = data_pop[data_pop['City'] == 'Zürich'].copy()
total_origin_zh = df_zh['origin'].sum()

df_zh['weight'] = df_zh['origin']/total_origin_zh
population_weight_zh = (pop_zh['habitant']+total_origin_zh) / (total_origin + population_total)
df_zh

,city,origin,weight
11,Zürich Oerlikon,113800,0.171437
19,Zürich HB,550000,0.828563


In [12]:
df_zh['weight_population'] = df_zh['weight']*population_weight_zh[0]
df_zh['destination'] = df_zh['weight_population']*total_origin

df_zh

,city,origin,weight,weight_population,destination
11,Zürich Oerlikon,113800,0.171437,0.045174,105643.939168
19,Zürich HB,550000,0.828563,0.218328,510581.428315


Menage the other city

In [13]:
data_red['weight'] = (data_red['population']+data_red['origin'])/(population_total + total_origin)

data_red['destination'] = data_red['weight']*total_origin
data_red.sort_values(by='destination')

,city,population,origin,weight,destination
11,Bellinzona,46544,16100,0.015123,35366.082669
8,Fribourg/Freiburg,37653,31500,0.016694,39040.781476
10,Neuchâtel,44597,28900,0.017743,41493.215278
15,Chur,38129,40300,0.018933,44277.608352
0,Thun,43670,40400,0.020295,47462.272045
7,Baden,32566,55400,0.021236,49661.784497
12,Zug,31345,58900,0.021786,50948.408953
6,Lugano,62464,34100,0.023311,54515.842009
14,Olten,30678,69600,0.024208,56612.605163
16,Aarau,22290,78600,0.024356,56958.113793


Concatenate and clean the Origin Destination

In [14]:
data_red = data_red.drop(['population', 'weight'], axis=1)
df_ge = df_ge.drop(['weight', 'weight_population'], axis=1)
df_zh = df_zh.drop(['weight','weight_population'], axis=1)

production = pd.concat([df_ge, df_zh], axis=0, ignore_index=True)  # Ignore original index and create a new one

In [15]:
prod_att = pd.concat([production, data_red], axis=0, ignore_index=True)  # Ignore original index and create a new one


Sanity Check

In [16]:
origin = prod_att['origin'].sum()
destination = prod_att['destination'].sum()
print(origin, destination)

2338600 2338600.0000000005


In [17]:
prod_att[['origin', 'destination']] = np.round(prod_att[['origin', 'destination']]).astype(int)

Save the dataframe

In [18]:
#od_df.to_csv('prod_att.csv', index=False)                # To CSV

In [19]:
prod_att


,city,origin,destination
0,Genève-Aéroport,43000,45734
1,Genève,187600,199531
2,Zürich Oerlikon,113800,105643
3,Zürich HB,550000,510581
4,Thun,40400,47462
5,Basel SBB,140900,177525
6,Lausanne,127900,152045
7,Winterthur,134700,142045
8,St. Gallen,77400,87128
9,Luzern,145400,129418


# Gravity Model

In [20]:
distance_matrix = pd.read_csv('data/distance_matrix.csv')
pa = prod_att.copy()

In [21]:
distance_matrix = distance_matrix.drop('Lugano Nord', axis=1)

# List of stations to remove
to_remove = ['Lugano Nord']

# Filter rows where 'Station_Name' is NOT in to_remove
distance_matrix = distance_matrix[~distance_matrix['Unnamed: 0'].isin(to_remove)]

# Optional: Reset index if needed
distance_matrix = distance_matrix.reset_index(drop=True)
distance_matrix

,Unnamed: 0,Bern,Basel SBB,Lausanne,Luzern,St. Gallen,Winterthur,Zug,Aarau,Baden,...,Fribourg/Freiburg,Genève-Aéroport,Genève,Neuchâtel,Olten,Thun,Bellinzona,Lugano,Zürich Oerlikon,Zürich HB
0,Bern,0.000,104798.677,96888.245,119979.281,202576.446,145437.404,141853.610,78999.684,110068.246,...,31109.895,163033.063,157114.875,62661.789,65626.280,31264.012,285352.211,314907.207,124018.454,120492.589
1,Basel SBB,104798.677,0.000,177474.735,94940.490,176122.563,118983.521,116814.819,52545.801,83614.363,...,135908.572,233259.129,227340.941,102386.778,39172.397,132299.575,260313.420,289868.416,97564.571,94038.706
2,Lausanne,96888.245,177474.735,0.000,216867.526,299464.691,242325.649,238741.855,175887.929,206956.491,...,65778.350,66144.818,60226.630,75087.957,162514.525,128152.257,382240.456,411795.452,220906.699,217380.834
3,Luzern,119979.281,94940.490,216867.526,0.000,139430.850,82291.808,28417.295,69141.497,78091.616,...,151089.176,275911.172,269992.984,145038.821,55768.093,147480.179,169866.354,199421.350,60872.858,57346.993
4,St. Gallen,202576.446,176122.563,299464.691,139430.850,0.000,57139.042,111013.555,123576.762,104671.032,...,233686.341,357093.245,351175.057,226220.894,136950.166,230077.344,304803.780,334358.776,78557.992,83926.409
5,Winterthur,145437.404,118983.521,242325.649,82291.808,57139.042,0.000,53874.513,66437.720,47531.990,...,176547.299,299954.203,294036.015,169081.852,79811.124,172938.302,247664.738,277219.734,21418.950,26787.367
6,Zug,141853.610,116814.819,238741.855,28417.295,111013.555,53874.513,0.000,68580.051,49674.321,...,172963.505,297785.501,291867.313,166913.150,77642.422,169354.508,193790.225,223345.221,32455.563,28929.698
7,Aarau,78999.684,52545.801,175887.929,69141.497,123576.762,66437.720,68580.051,0.000,31068.562,...,110109.579,233516.483,227598.295,102644.132,13373.404,106500.582,234514.427,264069.423,45018.770,41492.905
8,Baden,110068.246,83614.363,206956.491,78091.616,104671.032,47531.990,49674.321,31068.562,0.000,...,141178.141,264585.045,258666.857,133712.694,44441.966,137569.144,243464.546,273019.542,26113.040,22587.175
9,Biel/Bienne,33461.669,73186.658,104288.077,115838.701,197020.774,139881.732,137713.030,73444.012,104512.574,...,64571.564,160072.471,154154.283,29200.120,60070.608,60962.567,281211.631,310766.627,118462.782,114936.917


In [22]:
def clean_city_name(name):

    name = str(name)
    
    name = unicodedata.normalize('NFKD', name)\
                      .encode('ascii', 'ignore')\
                      .decode('utf-8')
    
    name = ''.join(char for char in name if char.isalnum()).lower()
    
    return name

def uniform_city_names(df, city_column):
    df = df.copy()
    
    df[city_column] = df[city_column].apply(clean_city_name)
    
    return df

In [23]:
distance_matrix = uniform_city_names(distance_matrix, 'Unnamed: 0')
pa =  uniform_city_names(pa, 'city')

pa.sort_values(by='destination',ascending=False)
pa_ord = pa.set_index('city').reindex(distance_matrix['Unnamed: 0']).reset_index()

In [24]:
pa_ord

,Unnamed: 0,origin,destination
0,bern,298900,244708
1,baselsbb,140900,177525
2,lausanne,127900,152045
3,luzern,145400,129418
4,stgallen,77400,87128
5,winterthur,134700,142045
6,zug,58900,50948
7,aarau,78600,56958
8,baden,55400,49661
9,bielbienne,65200,67899


Cost Matrix

In [25]:
cost_matrix = distance_matrix.iloc[:, 1:].to_numpy()

Gravity Model

In [26]:
O_totals = pa_ord['origin'].values
D_totals = pa_ord['destination'].values


In [27]:
beta = 0.1  # Calibrated parameter (adjust based on observed data)
def deterrence_function(cost, beta=0.1):
    return 1/cost  # Exponential form

In [28]:
counter = 0 # We need a counter to keep track of the first iteration
threshold_gravity = 0.000001 # The convergence threshold of the gravity model
threshold_MTL = 0.000001 # The convergence threshold of the MTL comparison
condition_MTL = 1 # condition for breaking the loop - must be larger than the threshold_MTL
diffA = np.array([1,1]) # we set the while loop condition to a large value

In [29]:
As = np.zeros(len(O_totals)) # The A values of the current iteration
As0 = np.zeros(len(O_totals)) # The A values of the previous iteration
Bs = np.ones(len(O_totals)) # The B values of the current iteration
OD_est = np.zeros((len(O_totals), len(O_totals))) # The estimated OD matrix

In [30]:
MTL = cost_matrix.mean()
MTL

np.float64(159953.77134240363)

In [31]:
n=1/MTL
GC = cost_matrix

In [32]:
while condition_MTL>threshold_MTL:

    ##############################################################
    ##############################################################

    # The gravity model part
    
    Fc = np.exp(-n*GC) # The deterrence function
    np.fill_diagonal(Fc,1e-6 )
    
    while min(diffA) > threshold_gravity:
        for i in range(len(As)):
            Sum_B = 0
            for j in range(len(As)):
                Sum_B += Bs[j]*D_totals[j]*Fc[i,j]
            As[i] = 1/Sum_B
        
        diffA = abs(As-As0)
        if min(diffA)<threshold_gravity:
            break
        
        for j in range(len(As)):
            Sum_A = 0
            for i in range(len(As)):
                Sum_A += As[i]*O_totals[i]*Fc[i,j]
            Bs[j] = 1/Sum_A
        As0 = As.copy()

    # Here the new OD matrix is computed
    
    for i in range(len(As)):
        for j in range(len(As)):
            OD_est[i,j] = As[i]*Bs[j]*O_totals[i]*D_totals[j]*Fc[i,j]

    # Reset some indicators
    As = np.zeros(len(O_totals)) # The A values of the current iteration
    As0 = np.zeros(len(O_totals)) # The A values of the previous iteration
    Bs = np.ones(len(O_totals)) # The B values of the current iteration        
    diffA = np.array([1,1])
    
    ##############################################################
    ##############################################################
    
    if counter==0: # Update of n in the first iteration only
        n0 = n
        numerator = OD_est*GC
        numerator = numerator.sum()
        denominator = (OD_est.sum())
        MTL_new = numerator/denominator
        MTL_new0 = MTL_new
        n = n*(MTL_new/MTL)
    
    if counter>0: # Update of n in later iterations
        n00=n
        numerator = OD_est*GC
        numerator = numerator.sum()
        denominator = (OD_est.sum())
        MTL_new = numerator/denominator
        
        n = ((MTL-MTL_new0)*n-(MTL-MTL_new)*n0)/(MTL_new-MTL_new0) # Parameter update step
        n0=n00
        MTL_new0 = MTL_new

    counter = counter + 1 # Increase the counter by 1

    condition_MTL = abs(MTL_new-MTL)
    
    if(condition_MTL<=threshold_MTL):
        break

    print(MTL_new)

108442.93778883578
118188.00444499025
172069.94157541494
158428.90866158705
159920.77988260955
159953.87687471433
159953.77133524235


In [33]:
OD_est = np.round(OD_est,0)
print(OD_est)

[[    0. 23586. 19779. 17906. 15032. 21037.  7473.  7064.  6692.  7456.
   8324.  4260.  7099. 30487.  4926.  6774.  5181.  7612. 12697. 14775.
  70741.]
 [14707.     0. 11096.  7576.  6336.  8867.  3162.  2977.  2820.  3750.
   3508.  2550.  3874. 16637.  2478.  2855.  3070.  3220.  5372.  6228.
  29817.]
 [11032.  9926.     0.  8039.  6749.  9445.  3355.  3171.  3004.  3122.
   3737.  1620.  1899.  8156.  1765.  3041.  2326.  3417.  5700.  6634.
  31760.]
 [15745. 10683. 12673.     0.  5905.  8264.  2567.  3199.  2857.  4321.
   3085.  2730.  4463. 19167.  2855.  3068.  3287.  2600.  4337.  5804.
  27790.]
 [ 8765.  5925.  7055.  3916.     0.  3450.  1429.  1652.  1369.  2396.
   1226.  1520.  2475. 10630.  1583.  1702.  1830.  1665.  2777.  2717.
  13320.]
 [15382. 10397. 12381.  6872.  4326.     0.  2508.  2899.  2403.  4205.
   2750.  2667.  4344. 18654.  2778.  2986.  3211.  2921.  4873.  4768.
  23375.]
 [ 6527.  4429.  5254.  2549.  2140.  2995.     0.  1249.  1035.  1791.
   1

Export

In [34]:
rows, cols = np.where(OD_est != 0)  
q_values = OD_est[rows, cols]     

df_long = pd.DataFrame({
    'org': rows,
    'dest': cols,
    'q': q_values
})

df_long = df_long.sort_values(by=['org', 'dest']).reset_index(drop=True)

print(df_long)

     org  dest        q
0      0     1  23586.0
1      0     2  19779.0
2      0     3  17906.0
3      0     4  15032.0
4      0     5  21037.0
..   ...   ...      ...
415   20    15  13798.0
416   20    16  14836.0
417   20    17  13498.0
418   20    18  22515.0
419   20    19  22559.0

[420 rows x 3 columns]


In [35]:
#df_long.to_csv('od_df.csv', index=False)  
